# Requirements

In [2]:
# Add as many imports as you need.

# Laboratory Exercise - Run Mode (8 points)

## Introduction
This laboratory assignment's primary objective is to fine-tune a pre-trained language model for binary classification on a dataset consisting of wine reviews. The dataset contains two attributes: **description** and **points**. The description is a brief text describing the wine and the points represent a quality metric ranging from 1 to 100. If some wine has at least 90 points it is considered **exceptional**. Your task involves predicting if some wine is **exceptional** based on its review.

## The Wine Reviews Dataset

Load the dataset using the `datasets` library.

In [7]:
# Write your code here. Add as many boxes as you need.
import pandas as pd

In [8]:
data = pd.read_csv('wine-reviews.csv')

In [9]:
data.head()

,description,points
0,"Translucent in color, silky in the mouth, this...",85
1,"On the palate, this wine is rich and complex, ...",92
2,The producer blends 57% Chardonnay from the Ma...,92
3,"Pure Baga in all its glory, packed with dry an...",93
4,Think of Subsídio as a contribution rather tha...,89


## Target Extraction
Extract the target **exceptional** for each wine review. If some wine has at least 90 points it is considered **exceptional**.

In [11]:
# Write your code here. Add as many boxes as you need.
data['exceptional'] = (data['points'] >= 90).astype(int)

In [12]:
data.head()

,description,points,exceptional
0,"Translucent in color, silky in the mouth, this...",85,0
1,"On the palate, this wine is rich and complex, ...",92,1
2,The producer blends 57% Chardonnay from the Ma...,92,1
3,"Pure Baga in all its glory, packed with dry an...",93,1
4,Think of Subsídio as a contribution rather tha...,89,0


## Dataset Splitting
Partition the dataset into training and testing sets with an 80:20 ratio.


In [14]:
# Write your code here. Add as many boxes as you need.
from sklearn.model_selection import train_test_split

In [15]:
input = data['description']  
target = data['exceptional']

X_train, X_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

## Tokenization
Tokenize the texts using the `AutoTokenizer` class.

In [17]:
# Write your code here. Add as many boxes as you need.
from transformers import AutoTokenizer

C:\Users\kolov\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
from datasets import Dataset
import torch

In [30]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

X_train_tokenized = tokenizer(list(X_train))
X_test_tokenized = tokenizer(list(X_test))

train_dataset = Dataset.from_dict({
    'input_ids': X_train_tokenized['input_ids'],
    'attention_mask': X_train_tokenized['attention_mask'],
    'labels': torch.tensor(y_train.values)  # Convert labels to tensor
})

test_dataset = Dataset.from_dict({
    'input_ids': X_test_tokenized['input_ids'],
    'attention_mask': X_test_tokenized['attention_mask'],
    'labels': torch.tensor(y_test.values)  # Convert labels to tensor
})

## Fine-tuning a Pre-trained Language Model for Classification
Fine-tune a pre-trained language model for classification on the given dataset.

Define the model using the `AutoModelForSequenceClassification` class.

In [32]:
# Write your code here. Add as many boxes as you need.
from transformers import AutoModelForSequenceClassification

In [34]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Define the traning parameters using the `TrainingArguments` class.

In [36]:
from transformers import TrainingArguments

In [38]:
# Write your code here. Add as many boxes as you need.
training_args = TrainingArguments(
    output_dir="trainer",
    eval_strategy="epoch",
    per_device_train_batch_size=8,  # batch size for training
    per_device_eval_batch_size=8,  # batch size for evaluation
    metric_for_best_model="f1",
)

Define the training using the `Trainer` class.

In [44]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

In [52]:
from transformers import DataCollatorWithPadding

In [54]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [56]:
# Write your code here. Add as many boxes as you need.
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

Fine-tune (train) the pre-trained lanugage model.

In [60]:
import torch

In [62]:
torch.cuda.is_available()

False

In [64]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [66]:
device

'cpu'

In [ ]:
# Write your code here. Add as many boxes as you need.
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
trainer.evaluate()

Use the trained model to make predictions for the test set.

In [ ]:
# Write your code here. Add as many boxes as you need.


Assess the performance of the model by using different metrics provided by the `scikit-learn` library.

In [ ]:
# Write your code here. Add as many boxes as you need.

# Laboratory Exercise - Bonus Task (+ 2 points)

Implement a simple machine learning pipeline to classify wine reviews as **exceptional** or not. Use TF-IDF vectorization to convert text into numerical features and train a logistic regression. Split the dataset into training and testing sets, fit the pipeline on the training data, and evaluate its performance using metrics such as precision, recall, and F1-score. Analyze the texts to find the most influential words or phrases associated with the **exceptional** wines. Use the coefficients from the logistic regression trained on TF-IDF features to identify the top positive and negative keywords for **exceptional** wines. Present these keywords in a simple table or visualization (e.g., bar chart).

In [ ]:
# Write your code here. Add as many boxes as you need.